# Rate Limiting Example

You may run into a situation where you are being rate limited by the GitHub API. For example in the below example, the user `fabplot` is one of the most active users on GitHub and has many contributions. We are not providing any date filter so we are trying to pull the whole history.


In [11]:
%%bash

prfiesta -u fabpot

INFO     PR Fiesta 🦜🥳                                                         
WARNING  🙇 You were rate limited by the GitHub API, try requesting less data.  
⠏ Searching https://api.github.com with type:pr author:fabpot


## Solution

When fetching very large ranges like this, it's better to split up the request and combine the data back. 

In this example, we are splitting up the requests by year by defining the last 3 years and then making a seperate call to the `GitHubCollector.collect` method. Each invocation returns back a `pd.DataFrame` which we store into a list. Then at the very end we leverage `pd.concat` to combine them into a single DataFrame.

In [41]:
%reload_ext jupyter_black

import logging
from datetime import datetime
from typing import List

import pandas as pd

from prfiesta.collectors.github import GitHubCollector

In [47]:
current_year = datetime.utcnow().year

year_range = [current_year - x for x in range(3)]

year_range

[2023, 2022, 2021]

In [57]:
github = GitHubCollector()

# We will collect each of the years results into this list
results: List[pd.DataFrame] = []

for year in year_range:
    after = datetime(year, 1, 1)
    before = datetime(year, 12, 31)

    temp = github.collect("fabpot", after=after, before=before)
    results.append(temp)

    print(f"Fetched {temp.shape[0]} rows for year {year}")

print(len(results))

Fetched 79 rows for year 2023
Fetched 230 rows for year 2022
Fetched 174 rows for year 2021
3


In [58]:
# Combine all the results back
all_results_frame = pd.concat(results)


print(all_results_frame.shape)
all_results_frame.head(3)

(483, 111)


,id,number,title,labels,state,locked,assignee,assignees,comments,created_at,updated_at,closed_at,author_association,active_lock_reason,draft,body,state_reason,score,user.login,user.id,user.node_id,user.avatar_url,user.gravatar_id,user.url,user.html_url,user.followers_url,user.following_url,user.gists_url,user.starred_url,user.subscriptions_url,user.organizations_url,user.repos_url,user.events_url,user.received_events_url,user.type,user.site_admin,milestone.url,milestone.html_url,milestone.labels_url,milestone.id,...,pull_request.diff_url,pull_request.patch_url,pull_request.merged_at,reactions.url,reactions.total_count,reactions.+1,reactions.-1,reactions.laugh,reactions.hooray,reactions.confused,reactions.heart,reactions.rocket,reactions.eyes,milestone,repository_name,url,repository_url,html_url,timeline_url,labels_url,comments_url,events_url,assignee.login,assignee.id,assignee.node_id,assignee.avatar_url,assignee.gravatar_id,assignee.url,assignee.html_url,assignee.followers_url,assignee.following_url,assignee.gists_url,assignee.starred_url,assignee.subscriptions_url,assignee.organizations_url,assignee.repos_url,assignee.events_url,assignee.received_events_url,assignee.type,assignee.site_admin
0,1720141599,50388,Release v6.3.0-RC1,"[{'id': 237003792, 'node_id': 'MDU6TGFiZWwyMzc...",closed,False,None,[],0,2023-05-22 17:41:17+00:00,2023-05-22 17:41:31+00:00,2023-05-22 17:41:24+00:00,MEMBER,None,False,**Changelog** (https://github.com/symfony/symf...,None,1.0,fabpot,47313,MDQ6VXNlcjQ3MzEz,https://avatars.githubusercontent.com/u/47313?v=4,,https://api.github.com/users/fabpot,https://github.com/fabpot,https://api.github.com/users/fabpot/followers,https://api.github.com/users/fabpot/following{...,https://api.github.com/users/fabpot/gists{/gis...,https://api.github.com/users/fabpot/starred{/o...,https://api.github.com/users/fabpot/subscriptions,https://api.github.com/users/fabpot/orgs,https://api.github.com/users/fabpot/repos,https://api.github.com/users/fabpot/events{/pr...,https://api.github.com/users/fabpot/received_e...,User,False,https://api.github.com/repos/symfony/symfony/m...,https://github.com/symfony/symfony/milestone/29,https://api.github.com/repos/symfony/symfony/m...,8553231.0,...,https://github.com/symfony/symfony/pull/50388....,https://github.com/symfony/symfony/pull/50388....,2023-05-22T17:41:24Z,https://api.github.com/repos/symfony/symfony/i...,0,0,0,0,0,0,0,0,0,NaN,symfony/symfony,https://api.github.com/repos/symfony/symfony/i...,https://api.github.com/repos/symfony/symfony,https://github.com/symfony/symfony/pull/50388,https://api.github.com/repos/symfony/symfony/i...,https://api.github.com/repos/symfony/symfony/i...,https://api.github.com/repos/symfony/symfony/i...,https://api.github.com/repos/symfony/symfony/i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,1640837797,49817,[Scheduler] Improve triggers performance when ...,"[{'id': 100079, 'node_id': 'MDU6TGFiZWwxMDAwNz...",closed,False,None,[],2,2023-03-26 09:32:21+00:00,2023-05-22 12:47:55+00:00,2023-05-22 12:47:55+00:00,MEMBER,None,False,| Q | A\r\n| ------------- | ---\r...,None,1.0,fabpot,47313,MDQ6VXNlcjQ3MzEz,https://avatars.githubusercontent.com/u/47313?v=4,,https://api.github.com/users/fabpot,https://github.com/fabpot,https://api.github.com/users/fabpot/followers,https://api.github.com/users/fabpot/following{...,https://api.github.com/users/fabpot/gists{/gis...,https://api.github.com/users/fabpot/starred{/o...,https://api.github.com/users/fabpot/subscriptions,https://api.github.com/users/fabpot/orgs,https://api.github.com/users/fabpot/repos,https://api.github.com/users/fabpot/events{/pr...,https://api.github.com/users/fabpot/received_e...,User,False,https://api.github.com/repos/symfony/symfony/m...,https://github.com/symfony/symfony/milestone/29,https://api.github.com/repos/symfony/symfony/m...,8553231.0,...,https://github.com/symfony/symfony/pull/49817....,https://github.com/symfony/symfony/pull/49817....,2023-05-22T12:47:54Z